# Portfolio Analysis

In this project, we seek to run through some common portfolio construction and asset allocation strategies to gain some insights into the usefulness of standard measures like sharpe ratio. The end goal is to test such strategies to a sufficient level of rigour that we can convincingly employ them into our portfolio rebalancing pipeline. I will start off by testing my own portfolio and the procedures should be generalizable. Let's get started!

## 0. Project Setup

##### Assets

1. Sea Limited (NYSE:'**SE**')
2. Bilibili Inc (NASDAQ:'**BILI**')
3. Pinduoduo Inc (NASDAQ:'**PDD**')
4. NVIDIA Corporation (NASDAQ:'**NVDA**') 
5. Unity Software Inc (NYSE:'**U**')
6. Peloton Inc (NASDAQ:'**PTON**')
7. Alphabet Inc (NASDAQ:'**GOOG**)
8. Amazon.com Inc (NASDAQ:'**AMZN**)
9. Microsoft Corporation (NASDAQ:'**MSFT**')
10. Facebook Inc (NASDAQ:'**FB**')
11. Intel Corporation (NASDAQ: '**INTC**')
12. Roku Inc (NASDAQ:'**ROKU**')
13. Roblox Corp (NYSE:'**RBLX**')
14. Blackrock Inc (NYSE:'**BLK**')
15. Berkshire Hathaway Inc (NYSE:'**BRK-B**')
16. Intellia Therapeutics Inc (NASDAQ:'**NTLA**')
17. Autodesk, Inc. (NASDAQ:'**ADSK**')
18. Draftkings Inc (NASDAQ:'**DKNG**')
19. Clearpoint Neuro Inc (NASDAQ:'**CLPT**')

##### Data

Daily adjusted closing prices from yahoo finance API 

##### Time period

1 year, 3 year, 5 year & 10 year.

In [152]:
# Load the required packages 
# Computation
import numpy as np 
from scipy import fftpack
# Plotting
import matplotlib.pyplot as plt
import mplcursors
import matplotlib.ticker as mtick
import seaborn as sns
# Data analysis
import pandas as pd
from sklearn import preprocessing
# Data source
import yfinance as yf

## 1. Data Analysis

In [153]:
# shortlisted stocks for portfolio analysis
s_list = 'SE BILI PDD NVDA PTON U INTC GOOG AMZN ROKU MSFT FB BLK BRK-B RBLX ADSK DKNG CLPT'
df_1y = yf.download(tickers = s_list, period = '1y', interval = '1d', group_by = 'ticker')
df_3y = yf.download(tickers = s_list, period = '3y', interval = '1d', group_by = 'ticker')
df_5y = yf.download(tickers = s_list, period = '5y', interval = '1d', group_by = 'ticker')
df_10y = yf.download(tickers = s_list, period = '10y', interval = '1d', group_by = 'ticker')

# Benchmark - S&P500
bench = '^GSPC'
bench_1y = yf.download(tickers = bench, period = '1y', interval = '1d')
bench_3y = yf.download(tickers = bench, period = '3y', interval = '1d')
bench_5y = yf.download(tickers = bench, period = '5y', interval = '1d')
bench_10y = yf.download(tickers = bench, period = '10y', interval = '1d')

[*********************100%***********************]  18 of 18 completed
[*********************100%***********************]  18 of 18 completed
[*********************100%***********************]  18 of 18 completed
[*********************100%***********************]  18 of 18 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [154]:
# select column for adjusted close prices
df_close_1y = df_1y.xs('Adj Close', level = 1, axis = 1)
df_close_3y = df_3y.xs('Adj Close', level = 1, axis = 1)
df_close_5y = df_5y.xs('Adj Close', level = 1, axis = 1)
df_close_10y = df_10y.xs('Adj Close', level = 1, axis = 1)
bench_close_1y = pd.DataFrame(data = bench_1y['Adj Close'], columns = ['Adj Close'])
bench_close_3y = pd.DataFrame(data = bench_3y['Adj Close'], columns = ['Adj Close'])
bench_close_5y = pd.DataFrame(data = bench_5y['Adj Close'], columns = ['Adj Close'])
bench_close_10y = pd.DataFrame(data = bench_10y['Adj Close'], columns = ['Adj Close'])

# adding column name to benchmark table
bench_close_1y = bench_close_1y.rename(columns = {'Adj Close':'S&P500'})
bench_close_3y = bench_close_3y.rename(columns = {'Adj Close':'S&P500'})
bench_close_5y = bench_close_5y.rename(columns = {'Adj Close':'S&P500'})
bench_close_10y = bench_close_10y.rename(columns = {'Adj Close':'S&P500'})
 
# reordering column names in portfolio 
s_order = ['SE', 'BILI', 'PDD', 'NVDA', 'U', 'PTON', 'GOOG', 'AMZN', 'MSFT', 'FB', 'INTC', 'ROKU', 'RBLX',  'BLK', 'BRK-B', 'ADSK', 'DKNG', 'CLPT']
df_close_1y = df_close_1y[s_order]
df_close_3y = df_close_3y[s_order]
df_close_5y = df_close_5y[s_order]
df_close_10y = df_close_10y[s_order]

# appending benchmark prices to portfolio dataframe 
df_close_1y = pd.concat([df_close_1y,bench_close_1y], axis = 1)
df_close_3y = pd.concat([df_close_3y,bench_close_3y], axis = 1)
df_close_5y = pd.concat([df_close_5y,bench_close_5y], axis = 1)
df_close_10y = pd.concat([df_close_10y,bench_close_10y], axis = 1)

# drop row if all values are NaN
df_close_1y.dropna(axis = 0, how = 'all', inplace = True)
df_close_3y.dropna(axis = 0, how = 'all', inplace = True)
df_close_5y.dropna(axis = 0, how = 'all', inplace = True)
df_close_10y.dropna(axis = 0, how = 'all', inplace = True)

# check the last 5 trading days
df_close_1y.tail(5)

,SE,BILI,PDD,NVDA,U,PTON,GOOG,AMZN,MSFT,FB,INTC,ROKU,RBLX,BLK,BRK-B,ADSK,DKNG,CLPT,S&P500
Date,,,,,,,,,,,,,,,,,,,
2021-08-18,318.609985,68.459999,80.959999,190.399994,117.919998,110.650002,2731.399902,3201.219971,290.730011,355.450012,52.189999,344.720001,83.459999,901.909973,286.359985,323.519989,52.099998,17.600000,4400.270020
2021-08-19,321.119995,64.330002,74.690002,197.979996,116.410004,107.800003,2738.270020,3187.750000,296.769989,355.119995,52.439999,343.350006,81.349998,903.780029,285.130005,330.350006,51.540001,17.020000,4405.799805
2021-08-20,309.329987,66.540001,77.290001,208.160004,122.839996,108.070000,2768.739990,3199.949951,304.359985,359.369995,52.009998,351.200012,82.769997,917.169983,285.269989,334.380005,52.009998,16.709999,4441.669922
2021-08-23,315.230011,70.120003,81.080002,219.580002,125.290001,106.589996,2821.989990,3265.870117,304.649994,363.350006,53.230000,358.000000,85.059998,927.859985,285.619995,336.760010,53.320000,18.030001,4479.529785
2021-08-24,311.869995,78.410004,99.120003,217.929993,125.070000,113.709999,2847.969971,3305.780029,302.619995,365.510010,53.810001,356.820007,89.239998,928.799988,286.130005,341.070007,56.470001,17.930000,4486.229980


In [224]:
i = df_close_1y.iloc[:,0].first_valid_index()
i

Timestamp('2020-08-25 00:00:00')

In [225]:
df_close_1y.iloc[:,0].loc[i]

154.1300048828125

In [227]:
# normalize the prices 
n = len(df_close_1y.columns)
for i in range(n): 
    a = df_close_1y.iloc[:,i].first_valid_index()
    df_close_1y_start = df_close_1y.iloc[:,i].loc[a]
    df_close_1y_norm.iloc[:,i] = (df_close_1y.iloc[:,i] - df_close_1y_start)/df_close_1y_start
    
    b = df_close_1y.iloc[:,i].first_valid_index()
    df_close_3y_start = df_close_3y.iloc[:,i].loc[b]
    df_close_3y_norm.iloc[:,i] = (df_close_3y.iloc[:,i] - df_close_3y_start)/df_close_3y_start
    
    c = df_close_1y.iloc[:,i].first_valid_index()
    df_close_5y_start = df_close_5y.iloc[:,i].loc[c]
    df_close_5y_norm.iloc[:,i] = (df_close_5y.iloc[:,i] - df_close_5y_start)/df_close_5y_start
    
    d = df_close_1y.iloc[:,i].first_valid_index()
    df_close_10y_start = df_close_10y.iloc[:,i].loc[d]
    df_close_10y_norm.iloc[:,i] = (df_close_10y.iloc[:,i] - df_close_10y_start)/df_close_10y_start


In [228]:
# plotting out the prices
%matplotlib widget

# plot configurations
sns.set(style="darkgrid", font_scale=0.8)
palette = sns.color_palette("hls", 19)
fig, ax = plt.subplots(figsize=(8, 4))

# plotting out the figure
plot1 = sns.lineplot(ax=ax, data = df_close_1y_norm, dashes = False, palette=palette)
plt.legend(bbox_to_anchor=(1, 1), loc=2, borderaxespad=0., fontsize = 8)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.title('1 Year Returns')

# make it interactive
cursor = mplcursors.cursor(plot1, hover=True)
@cursor.connect("add")
def on_add(sel):
    sel.annotation.set(text=tt[sel.target.index])
    
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [229]:
# check the dimensions 
df_close_1y.shape

(252, 19)

In [230]:
# convert the price into rate of return
df_return_1y = df_close_1y.pct_change()
df_return_3y = df_close_3y.pct_change()
df_return_5y = df_close_5y.pct_change()
df_return_10y = df_close_10y.pct_change()

# check the descriptive statistics of 1 year rate of return
df_return_1y.describe()

,SE,BILI,PDD,NVDA,U,PTON,GOOG,AMZN,MSFT,FB,INTC,ROKU,RBLX,BLK,BRK-B,ADSK,DKNG,CLPT,S&P500
count,251.000000,251.000000,251.000000,251.000000,234.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,116.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000
mean,0.003443,0.003188,0.001575,0.002507,0.003471,0.003095,0.002418,0.000118,0.001492,0.001267,0.000660,0.004290,0.003169,0.002003,0.001230,0.001397,0.002217,0.007656,0.001099
std,0.035560,0.048252,0.047030,0.027016,0.042231,0.043471,0.016685,0.018273,0.015502,0.020922,0.020822,0.039713,0.045662,0.015211,0.010997,0.019699,0.039012,0.051388,0.009466
min,-0.103195,-0.170397,-0.122561,-0.092775,-0.141312,-0.202853,-0.054630,-0.075649,-0.061947,-0.063099,-0.105751,-0.124132,-0.123584,-0.046489,-0.029872,-0.073694,-0.084906,-0.165829,-0.035288
25%,-0.017390,-0.027267,-0.025790,-0.011649,-0.023977,-0.020195,-0.005641,-0.009539,-0.005965,-0.011875,-0.010087,-0.019520,-0.026788,-0.007161,-0.005430,-0.009454,-0.023578,-0.022805,-0.003691
50%,0.005105,0.001164,-0.002620,0.003161,0.000207,0.003589,0.002454,0.000824,0.000978,-0.000224,0.000557,0.000840,0.000817,0.002676,0.001228,0.002927,-0.000399,0.002171,0.001276
75%,0.026284,0.032733,0.021845,0.018765,0.029259,0.029069,0.010333,0.010454,0.010830,0.014134,0.012008,0.023318,0.029400,0.011607,0.007482,0.014841,0.024547,0.035301,0.007386
max,0.108809,0.221735,0.222496,0.080333,0.162555,0.144746,0.073961,0.063230,0.048249,0.083227,0.069684,0.176669,0.213281,0.040412,0.060570,0.047380,0.172697,0.164811,0.023791


### Rate of Return

In [231]:
# compute the annualized average rate of return across a 1 year, 3 year, 5 year & 10 year horizon.
nday = 252 # number of trading days
df_return_1y_mean = pd.DataFrame(data = df_return_1y.mean(axis = 0), columns = ['1Y Annual Return']) * nday
df_return_3y_mean = pd.DataFrame(data = df_return_3y.mean(axis = 0), columns = ['3Y Annual Return']) * nday
df_return_5y_mean = pd.DataFrame(data = df_return_5y.mean(axis = 0), columns = ['5Y Annual Return']) * nday
df_return_10y_mean = pd.DataFrame(data = df_return_10y.mean(axis = 0), columns = ['10Y Annual Return']) * nday
data_frames = [df_return_1y_mean, df_return_3y_mean, df_return_5y_mean, df_return_10y_mean]
df_return_merge = pd.concat(data_frames, axis = 1)

#df_return_10y_mean.plot.barh()
df_return_merge

,1Y Annual Return,3Y Annual Return,5Y Annual Return,10Y Annual Return
SE,0.867752,1.187131,0.943817,0.943817
BILI,0.803294,0.843755,0.782829,0.782829
PDD,0.396957,0.805167,0.684384,0.684384
NVDA,0.631770,0.517377,0.649876,0.517223
U,0.874691,0.874691,0.874691,0.874691
PTON,0.779900,1.030317,1.030317,1.030317
GOOG,0.609215,0.324675,0.298172,0.272297
AMZN,0.029714,0.234684,0.338250,0.333311
MSFT,0.376044,0.402375,0.383618,0.306195
FB,0.319362,0.307540,0.270858,0.311551


We see that the annualized returns are skewed to the right due to the broad spike in equity valuations in 2020. Also, some of the companies in the list only went public in the last 5 years, which explains the potentially transitory outperformance against the benchmark. A subtle takeaway is that some of the returns are largely driven by a particular breakout year (e.g. AMZN from March to September 2020), which serves as a timely caution against making large portfolio adjustments based on 1-year performance alone. 

### Variance

In [232]:
# compute the daily standard deviation across a 1 year, 3 year, 5 year & 10 year horizon.
df_std_1y = pd.DataFrame(data = df_close_1y_norm.std(axis = 0), columns = ['1Y Daily Standard Deviation'])  
df_std_3y = pd.DataFrame(data = df_close_3y_norm.std(axis = 0), columns = ['3Y Daily Standard Deviation']) 
df_std_5y = pd.DataFrame(data = df_close_5y_norm.std(axis = 0), columns = ['5Y Daily Standard Deviation']) 
df_std_10y = pd.DataFrame(data = df_close_10y_norm.std(axis = 0), columns = ['10Y Daily Standard Deviation']) 
data_frames = [df_std_1y, df_std_3y, df_std_5y, df_std_10y]
df_std_merge = pd.concat(data_frames, axis = 1)

# compute the daily variance across a 1 year, 3 year, 5 year & 10 year horizon.
df_var_1y = pd.DataFrame(data = df_close_1y_norm.var(axis = 0), columns = ['1Y Daily Variance'])  
df_var_3y = pd.DataFrame(data = df_close_3y_norm.var(axis = 0), columns = ['3Y Daily Variance']) 
df_var_5y = pd.DataFrame(data = df_close_5y_norm.var(axis = 0, skipna = True), columns = ['5Y Daily Variance']) 
df_var_10y = pd.DataFrame(data = df_close_10y_norm.var(axis = 0, skipna = True), columns = ['10Y Daily Variance']) 
data_frames = [df_var_1y, df_var_3y, df_var_5y, df_var_10y]
df_var_merge = pd.concat(data_frames, axis = 1)

df_var_merge

,1Y Daily Variance,3Y Daily Variance,5Y Daily Variance,10Y Daily Variance
SE,0.096534,0.375812,0.349846,0.349846
BILI,0.444966,0.655820,0.622315,0.622315
PDD,0.154395,0.324519,0.323207,0.323207
NVDA,0.041867,0.150004,0.128649,0.128697
U,0.108145,0.108145,0.108145,0.108145
PTON,0.091000,0.422835,0.422835,0.422835
GOOG,0.063349,0.086141,0.083390,0.107062
AMZN,0.002353,0.043619,0.065061,0.086119
MSFT,0.015574,0.066681,0.093361,0.101187
FB,0.015255,0.047204,0.043873,0.084166


In [233]:
# compute the pearson pairwise correlation matrix
df_return_1y_corr = df_close_1y_norm.corr(method='pearson')
df_return_3y_corr = df_close_3y_norm.corr(method='pearson')
df_return_5y_corr = df_close_5y_norm.corr(method='pearson')
df_return_10y_corr = df_close_10y_norm.corr(method='pearson')

In [234]:
# plotting the 1 year correlation matrix 

sns.set_theme(style="white")
# generate a mask for the upper triangle
mask = np.triu(np.ones_like(df_return_1y_corr, dtype=bool))

# set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# generate a custom diverging colormap
cmap = sns.diverging_palette(240, 5, as_cmap=True)

# draw the heatmap with the mask and correct aspect ratio
sns.heatmap(df_return_1y_corr, mask=mask, cmap=cmap, vmax=1, center=0, vmin = -1,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

This result is sort of expected. In general, we know that the big tech should be highly correlated with the S&P500, and the so-called "high-beta/ARK" stocks should correlate with each other. BRK-B has a "defensive" reputation so it makes sense that it would have zero to negative correlation with the mid-cap tech stocks. Lastly, we note that the textbook diversification of idiosyncratic risks seems elusive within an all equity portfolio.

In [235]:
# compute the pairwise variance covariance matrix 
df_return_1y_cov = df_close_1y_norm.cov() 
df_return_3y_cov = df_close_3y_norm.cov()
df_return_5y_cov = df_close_5y_norm.cov()
df_return_10y_cov = df_close_10y_norm.cov()

### Risk-Return Measures

In [248]:
# Sharpe ratio 
df_sharpe_merge = np.divide(df_return_merge,df_std_merge)
df_sharpe_merge.rename(columns = {'1Y Annual Return':'1Y Sharpe', '3Y Annual Return':'3Y Sharpe', '5Y Annual Return':'5Y Sharpe','10Y Annual Return':'10Y Sharpe'}, inplace=True)
df_sharpe_merge

,1Y Sharpe,3Y Sharpe,5Y Sharpe,10Y Sharpe
SE,2.792907,1.936480,1.595693,1.595693
BILI,1.204235,1.041896,0.992343,0.992343
PDD,1.010244,1.413403,1.203814,1.203814
NVDA,3.087611,1.335844,1.811873,1.441760
U,2.659811,2.659811,2.659811,2.659811
PTON,2.585350,1.584475,1.584475,1.584475
GOOG,2.420470,1.106229,1.032548,0.832196
AMZN,0.612523,1.123687,1.326099,1.135798
MSFT,3.013266,1.558225,1.255498,0.962575
FB,2.585660,1.415509,1.293138,1.073894


Comparing the sharpe ratios of the individual stocks against the benchmark, it becomes apparent that beating the market on a risk-adjusted basis is no simple feat (at least when we use volatility as a risk measure). 

## 2. Portfolio Optimization

### Current Allocation

In [237]:
# current weightage 
arr_w = np.array([[0.30,0.14,0.10,0.08,0.08,0.04,0.04,0.04,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.01]])
df_w = pd.DataFrame(data = arr_w, columns = s_order)
df_w = df_w.rename(index={0: "weight"})
df_w

,SE,BILI,PDD,NVDA,U,PTON,GOOG,AMZN,MSFT,FB,INTC,ROKU,RBLX,BLK,BRK-B,ADSK,DKNG,CLPT
weight,0.3,0.14,0.1,0.08,0.08,0.04,0.04,0.04,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.01


In [165]:
arr_w.sum()

1.0

In [239]:
# dropping benchmark from dataframe
df_return_1y_mean = df_return_1y_mean.drop(index = ['S&P500'])

df_return_1y_cov = df_return_1y_cov.drop(columns= ['S&P500'])
df_return_1y_cov = df_return_1y_cov.drop(index= ['S&P500'])

In [240]:
# convert dataframe back to numpy array for ease of manipulation
np_return_1y_mean = df_return_1y_mean.T.to_numpy()

np_return_1y_cov = df_return_1y_cov.to_numpy()

In [255]:
# portfolio return 
portfolio_return_1y = np.inner(np_return_1y_mean,arr_w)
portfolio_var_1y = np.matmul(np.matmul(arr_w,np_return_1y_cov),arr_w.T)
portfolio_sharpe_1y = portfolio_return_1y/np.sqrt(portfolio_var_1y)
benchmark_sharpe_1y = df_sharpe_merge['1Y Sharpe']['S&P500']
print(f'Portfolio 1y return:{portfolio_return_1y}; Portfolio 1y variance:{portfolio_var_1y}')
print(f'Portfolio 1y sharpe ratio:{portfolio_sharpe_1y}; S&P500 1y Sharpe: {benchmark_sharpe_1y}')

Portfolio 1y return:[[0.6930387]]; Portfolio 1y variance:[[0.069492]]
Portfolio 1y sharpe ratio:[[2.62899702]]; S&P500 1y Sharpe: 2.717805773839361


Here, we see the diversification effects of a multi-asset portfolio. While the 1 year portfolio returns remains at a respectable 69%, the portfolio volatility is drastically reduced as compared to the volatility of the individual stocks. In MPT terminology, the idiosyncratic risks of individual assets is reduced.

However, we once again see the power of the S&P500, which boosts a Sharpe ratio of 2.72 compared to our current portfolio Sharpe ratio of 2.63. Now let's see if we can use the MPT techniques to construct a portfolio that outperforms the market on a risk-adjusted basis. 

### Minimum Variance Portfolio 

Given a portfolio of $n$ assets and a required portfolio rate of return $\mu_{p}$, find $w_{i}$ for $i \in [1:n]$ such that the portfolio variance $\sigma_{p}$ is minimized. 

### Maximum Return Portfolio

Given a portfolio of $n$ assets and a maximum portfolio variance  $\sigma_{p}$, find $w_{i}$ for $i \in [1:n]$ such that the portfolio rate of return $\mu_{p}$ is minimized. 